# PyTorch distributed training on SageMaker

## 1 说明
本章内容为用SageMaker进行训练，数据来自MNIST。

## 2 运行环境
Kernel 选择pytorch_latest_p36。  
本文在boto3 1.20.23和sagemaker 2.70.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

如果版本较低，请执行以下命令，重启kernal后再检查版本

In [ ]:
!pip install -U boto3 -i https://opentuna.cn/pypi/web/simple/

In [ ]:
!pip install -U sagemaker -i https://opentuna.cn/pypi/web/simple/

## 3 获取Role

In [ ]:
import boto3
import sagemaker

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题，需要先打开https://cn-northwest-1.console.amazonaws.cn/sagemaker/home?region=cn-northwest-1#/notebook-instances/create以创建IAM Role
print(role)

## 4 数据
### 4.1 获取数据

In [ ]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

### 4.2 上传数据到 S3

SageMaker训练时，数据从S3获取

In [ ]:
#修改为自己的bucket
bucket='junzhong'
prefix = 'sagemaker/DEMO-pytorch-mnist'

In [ ]:
!aws s3 sync data/ s3://$bucket/$prefix/

In [ ]:
inputs='s3://{}/{}/'.format(bucket, prefix)
inputs

## 5 训练

### 5.1 训练脚本
code/mnist.py脚本提供了训练和推理SageMaker模型所需的所有代码（model_fn函数用于推理时加载模型）。

### 5.2 在SageMaker里训练
使用SageMaker训练时，环境变量已设置以下值，不用再在程序里指定
```
NCCL_SOCKET_IFNAME=eth0
MASTER_ADDR=algo-1
MASTER_PORT=7777
```

In [ ]:
#是否使用spot实例
use_spot = True

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    source_dir="code",
                    role=role,
                    framework_version='1.7',
                    py_version='py3',
                    instance_count=2,
                    instance_type='ml.p3.2xlarge',#CPU:ml.m5.xlarge,GPU:ml.p3.2xlarge
                    use_spot_instances=use_spot,
                    max_wait=7200 if use_spot else None,
                    max_run=7200,
                    hyperparameters={
                        'epochs': 10,
                        'backend': 'nccl',#CPU使用gloo，GPU使用nccl
                    })

In [ ]:
estimator.fit({'training': inputs})

## 6 下载model(可选)

In [ ]:
!aws s3 cp s3${estimator.model_data} ./

In [ ]:
!tar zxf model.tar.gz

## 7 部署推理服务
在code/mnist.py脚本中实现了所需的model_fn。我们将使用sagemaker pytorch容器中定义的input_fn, predict_fn, output_fn的默认实现。  
https://github.com/aws/deep-learning-containers/blob/master/pytorch/inference/docker/build_artifacts/default_inference_handler.py  
该步骤大概需要7分钟
### 方式1：使用默认model
注意：部署时的model.tar.gz不是上一步的model.tar.gz。这里的model包含训练时的本地code目录  

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large',endpoint_name="mnist")

### 方式2：使用自定义model
自定义model需按如下格式进行组织
```
model.tar.gz/
|- model.pth
|- code/
  |- inference.py
  |- requirements.txt
```

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model_uri = "s3://junzhong/sagemaker/model.tar.gz"
my_model = PyTorchModel(
            model_data=model_uri, 
            role=role,
            entry_point="mnist.py",
            framework_version='1.6.0',
            py_version='py3')

predictor = my_model.deploy(initial_instance_count=1, instance_type='ml.m5.large',endpoint_name="mnist2")

## 8 评估

In [ ]:
import gzip 
import numpy as np
import random
import os

data_dir = 'data/MNIST/raw'
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

In [ ]:
image_size = 3
mask1 = random.sample(range(len(images)), image_size) # randomly select some of the test images
mask2 = np.array(mask1, dtype=np.int)
data = images[mask2]

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(2,2))
for index, mask in enumerate(mask1):
    plt.subplot(1,image_size,index+1)
    plt.axis('off')
    plt.imshow(images[mask])

重启kernel后，重新获取predictor；未重启kernal，不用执行

In [ ]:
'''
from sagemaker.pytorch.model import PyTorchPredictor
endpoint_name = "mnist"
predictor = PyTorchPredictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session())
'''

推理输入内容是NumPy array或Python list  
默认推理结果是一个NumPy array

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
for i in range(0,image_size):
    print("Most likely answer: {}".format(np.argmax(response[i])))

## 8 清理

In [ ]:
predictor.delete_endpoint()